# KBO에서 가져온 데이터 가공

In [1]:
#-*- coding: utf-8 -*-
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
import numpy as np
import pickle

In [15]:
f = open('Parsing_TeamData_from_KBO.txt', 'rb')

result = pickle.load(f)

f.close()

In [17]:
### 한글 팀명 -> 영어 팀명 ###

j, k = 0, -1
for i in range(len(result)):
    if(result[i][2][j]=='삼성'):
        result[i][2][j] = 'SAM'
    elif(result[i][2][j]=='한화'):
        result[i][2][j] = 'HAN'
    elif(result[i][2][j]=='두산'):
        result[i][2][j] = 'DUS'
    elif(result[i][2][j]=='롯데'):
        result[i][2][j] = 'LOT'
    elif(result[i][2][j]=='넥센'):
        result[i][2][j] = 'NEX'
for i in range(len(result)):
    if(result[i][2][k]=='삼성'):
        result[i][2][k] = 'SAM'
    elif(result[i][2][k]=='한화'):
        result[i][2][k] = 'HAN'
    elif(result[i][2][k]=='두산'):
        result[i][2][k] = 'DUS'
    elif(result[i][2][k]=='롯데'):
        result[i][2][k] = 'LOT'
    elif(result[i][2][k]=='넥센'):
        result[i][2][k] = 'NEX'

In [18]:
### vs제거 ###

for i in range(len(result)):
    for j in range(len(result[i][2])):
        if result[i][2][j] == 'vs':
            del result[i][2][j]
            break

In [19]:
### 한화의 각 경기 당 점수차(2011.04~2016.10) ###

score = []

for i in range(len(result)):
    if (len(result[i][2])>3 and result[i][2][0] == 'HAN'):
        score.append(int(result[i][2][1])-int(result[i][2][2]))
    elif len(result[i][2])>3 and result[i][2][0] != 'HAN':
        score.append(int(result[i][2][2])-int(result[i][2][1]))
    #elif len(result[i][2])<3:
        #score.append('NULL')

In [32]:
## train데이터, label데이터 생성 ##

train = []
label = []

datalen = 30

for i in range(len(score)):
    if i+datalen>len(score):
        continue
    train.append([])
    for j in range(datalen):  
        train[i].append(score[i+j])
        
i=1
while(1):
    if i==len(train):
        break
    label.append([])
    if train[i][0]<0:
        label[i-1].append(0)
        label[i-1].append(0)
        label[i-1].append(1)
    elif train[i][0]>0:
        label[i-1].append(1)
        label[i-1].append(0)
        label[i-1].append(0)
    else:
        label[i-1].append(0)
        label[i-1].append(1)
        label[i-1].append(0)
    i = i+1
        

print len(train)
print len(label)

912
911


In [21]:
## numpyarray 변환 ##

nptrain = np.array(train)
nplabel = np.array(label)

In [22]:
f1 = open('TrainData_team_from_KBO.txt', 'wb')
f2 = open('LabelData_team_from_KBO.txt', 'wb')
pickle.dump(nptrain, f1)
pickle.dump(nplabel, f2)
f1.close()
f2.close()

In [41]:
## 한화의 게임 날짜 ##

date = []
for i in range(len(result)):
    if len(result[i][2])>3:
        date.append(result[i][0][:10])
    
for i in range(len(date)):
    date[i] = date[i].replace(".", "-")

In [42]:
f = open('Data_HanHwa_GameDate_from_KBO.txt', 'wb')

pickle.dump(date, f)

f.close()